In [ ]:
# Import Python standard library and IPython packages we need.
import os
import subprocess
import sys
import json
from pathlib import Path

# Ask GRASS GIS where its Python packages are.
sys.path.append(
    subprocess.check_output(["grass", "--config", "python_path"], text=True).strip()
)

# Import the GRASS GIS packages we need.
import grass.script as gs
import grass.jupyter as gj

In [ ]:
grass_project = "data/ww_covid_2022/"
subproject = "anonymized_data3"

In [ ]:
!grass -e -c $grass_project/$subproject

In [ ]:
session = gj.init(Path(grass_project) / subproject)

In [ ]:
!v.extract input=raleigh_gravity_mains@imported_RaleighSewer where="Discharge =  'Smith Creek'" output=mains_subset

In [ ]:
!v.build.polylines input="mains_subset" output="mains_subset_polylines" cats="first" type="line,boundary"
!v.split input="mains_subset_polylines" layer="-1" output="mains_subset_dense" length=75 units="map"

In [ ]:
# Needs to be updated to 1) remove ascii 2) densified lines.
!v.out.ascii mains_subset separator=" " type=line format=standard | grep -vE "^L|^ 1" | grep -v "[a-zA-Z]" | v.in.ascii input=- output=mains_points format=point separator=" " x=2 y=3 --o
!v.kernel input="mains_points" output="mains_density" radius=1000 dsize=0. segmax=100. distmax=100. multiplier=1000 kernel="gaussian"

In [ ]:
gs.mapcalc(
    "mains_density_randomized = 2 * rand(mains_density, 1 / mains_density)"
    " + 1000 * sin(row() / 100)"
    " + 100 * sin(col())",
    seed=1,
)

In [ ]:
!v.move.points input=mains_subset_dense x_raster=mains_density_randomized y_raster=mains_density_randomized output=mains_subset_moved

In [ ]:
gs.run_command("g.region", vector="mains_subset_dense,mains_subset")
sewer_map = gj.Map(use_region=True, width=900)
sewer_map.d_background(color="white")
sewer_map.d_vect(map="mains_subset_dense", color="blue", legend_label="Original")
#sewer_map.d_vect(map="mains_subset_moved", color="red", legend_label="Shifted")
sewer_map.d_legend_vect(at=(70,10))
sewer_map.show()

In [ ]:
!v.info mains_subset_moved

In [ ]:
!ls -lah $grass_project/$subproject/vector/mains_subset_moved